# Projeto 1 - Problema N-Rainhas  
### Autor: Marcos Angelo Cemim
  
(i) Operadores que poderão ser utilizados:
  
- Cruzamento:  
 -- cxOnePoint()  
 -- cxTwoPoint()  
 -- cxUniform()  
  
- Mutação:  
 -- mutShuffleIndexes()  
 -- mutUniformInt()  
  
- Seleção:  
 -- selTournament()  
 -- selRoulette()  
  
(ii) Formas de execução:  
  

- Rodar pelo menos 2 (duas) combinações diferentes entre os operadores (Seleção, Cruzamento e Mutação) 
- Apresentar as soluções obtidas para cada combinação.   
- Desenvolver o Algoritmo Genético com o framework DEAP
- Executar as combinações  para 10, 15 e 20 Rainhas 

## Encontrando todas as combinações possíveis dos parâmetros

In [1]:
parameters_list = {
    'p1': ['OnePoint', 'TwoPoints', 'Uniform'],     # Cruzamentos
    'p2': ['Shuffleindexes', 'UniformInt'],         # Mutação
    'p3': ['Tournament', 'Roulette']                # Seleção
}

In [2]:
from itertools import product

def encontrar_combinacoes(dicionario):
    chaves = dicionario.keys()
    valores = [dicionario[chave] for chave in chaves]
    todas_combinacoes = list(product(*valores))
    return todas_combinacoes

In [3]:
lista_parametros = encontrar_combinacoes(parameters_list)

## Importação das bibliotecas

In [4]:
import random
from deap import base, creator, tools, algorithms
import warnings
import numpy as np
warnings.filterwarnings("ignore")
import time

## Implementação do algoritmo

#### Funçao de avaliação (fitness)

In [5]:
# Defina a função de aptidão
def fitness_function(solucao):
        h = 0
        #Contagem de ataques na diagonal e vertical
        for i in range(0, len(solucao)):
            for j in range(0, len(solucao)):
                if j > i:
                    # Avalia a diferença entre as colunas e as posições
                    # das rainhas dentro da coluna
                    if abs(i - j) == abs(solucao[i] - solucao[j]):
                       #  print(f'{i} - {j} - {solucao[i]}-{solucao[j]}')
                       h += 1
                    # Ataques por linha (horizontal)
                    # Avalia apenas as posições das rainhas
                    if abs(solucao[i] - solucao[j]) == 0:
                       h += 1
        return h,

# Defina a função de aptidão "inversa"
# Por definição, a função de seleçao "selRoulette" nãqo pode ser utilizada para minimização.
# Portanto, vamos criar uma função "inversa" para avaliar
def fitness_function_inv(solucao):
        h = 0
        melhorFitness = (len(solucao) * (len(solucao) - 1) // 2)
        #Contagem de ataques na diagonal e vertical
        for i in range(0, len(solucao)):
            for j in range(0, len(solucao)):
                if j > i:
                    # Avalia a diferença entre as colunas e as posições
                    # das rainhas dentro da coluna
                    if abs(i - j) == abs(solucao[i] - solucao[j]):
                       #  print(f'{i} - {j} - {solucao[i]}-{solucao[j]}')
                       h += 1
                    # Ataques por linha (horizontal)
                    # Avalia apenas as posições das rainhas
                    if abs(solucao[i] - solucao[j]) == 0:
                       h += 1
        return melhorFitness - h,
    

#### Parâmetros gerais (dfefinidos arbitrariamente)

In [6]:
pop_size = 100  # Tamanho da população
cxpb = 0.6      # Probabilidade de cruzamento
mutpb = 0.1     # Probabilidade de mutação
ngen = 2000     # Número de gerações
tourn_size = 3  # Indivíduos participantes do 'torunament'
indpb = 0.05    # Independent probability for each attribute to be exchanged to another position.

#### Rodando todas as combinações em todos os cenários sugeridos para avaliar os melhores desempenhos

In [7]:
for N in [10, 15, 20]:
    best_fit = 10000
    best_sol = ''
    best_param = ''
    print(f"{'-'*50} {N} RAINHAS {'-'*50}")
    for param in lista_parametros:
        inicio = time.time()
        print(f'{str(param):45s}', end= ' | ')
        # Configurar a estrutura DEAP para minimizar
        
        # Gera o objetivo toolbox responsável por registrar as configurações do framewrok
        toolbox = base.Toolbox()


        if param[2] == 'Roulette':
            # Cria o tipo de função fitness e indivíduo
            creator.create("maximizar", base.Fitness, weights=(1.0,))
            creator.create("individual", list, fitness=creator.maximizar)
            
        else:
            # Cria o tipo de função fitness e indivíduo
            creator.create("minimizar", base.Fitness, weights=(-1.0,))
            creator.create("individual", list, fitness=creator.minimizar)

        # Registra os nomes e os tipos de individuo, fitness e população
        toolbox.register("atributo", random.randint, 0, 1)
        toolbox.register("solucaoFinal", tools.initRepeat, creator.individual, toolbox.atributo, n=N )
        toolbox.register("Populacao", tools.initRepeat, list, toolbox.solucaoFinal)


        # Define a criação de indivíduos e populações
        toolbox = base.Toolbox()
        toolbox.register("permutation", random.sample, range(N), N)
        toolbox.register("individual", tools.initIterate, creator.individual, toolbox.permutation)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Definir operadores genéticos
        ## Cruzamento
        if param[0] == 'OnePoint':
            toolbox.register("mate", tools.cxOnePoint)    
        elif param[0] == 'TwoPoints':
            toolbox.register("mate", tools.cxTwoPoint)  
        elif param[0] == 'Uniform':
            toolbox.register("mate", tools.cxUniform, indpb=indpb)

        ## Mutação
        if param[1] == 'Shuffleindexes':
            toolbox.register("mutate", tools.mutShuffleIndexes, indpb=indpb)  
        elif param[1] == 'UniformInt':
            toolbox.register("mutate", tools.mutUniformInt, low=0, up=N, indpb=indpb)  

        ## Seleção
        if param[2] == 'Tournament':
            toolbox.register("select", tools.selTournament, tournsize=tourn_size)    # Seleção
            toolbox.register('evaluate', fitness_function)
        elif param[2] == 'Roulette':
            toolbox.register("select", tools.selRoulette) 
            toolbox.register('evaluate', fitness_function_inv)

        pop = toolbox.population(n=pop_size)
        hof = tools.HallOfFame(10)
        stat = tools.Statistics(lambda ind: ind.fitness.values)
        stat.register("Melhor Solucao", np.min)
        stat.register("Media da Populacao", np.mean)
        finalPop, log = algorithms.eaSimple(pop, toolbox, cxpb, mutpb, ngen, stat, hof, verbose = False)
        best_solution = tools.selBest(finalPop, 1)[0]
        new_fit = fitness_function(best_solution)[0]
        best_sol = best_solution if new_fit < best_fit else best_sol
        best_param = str(param) if new_fit < best_fit else best_param
        best_fit = new_fit if new_fit < best_fit else best_fit
        tempo = time.time()-inicio
        print(f"{tempo:10.2f} s | Melhor solução: {best_solution} @ {new_fit}")
    print("-"*180)
    print(f"Melhor solução: {best_param} com {best_fit} ataques")

-------------------------------------------------- 10 RAINHAS --------------------------------------------------
('OnePoint', 'Shuffleindexes', 'Tournament')  |       4.70 s | Melhor solução: [2, 0, 9, 6, 4, 9, 1, 8, 5, 7] @ 1
('OnePoint', 'Shuffleindexes', 'Roulette')    |       8.28 s | Melhor solução: [5, 8, 0, 7, 7, 2, 8, 3, 9, 3] @ 4
('OnePoint', 'UniformInt', 'Tournament')      |       4.69 s | Melhor solução: [9, 6, 8, 1, 4, 7, 0, 10, 5, 2] @ 0
('OnePoint', 'UniformInt', 'Roulette')        |       8.42 s | Melhor solução: [0, 9, 9, 6, 10, 2, 5, 1, 4, 7] @ 2
('TwoPoints', 'Shuffleindexes', 'Tournament') |       4.79 s | Melhor solução: [5, 9, 4, 1, 8, 6, 2, 0, 7, 9] @ 1
('TwoPoints', 'Shuffleindexes', 'Roulette')   |       8.40 s | Melhor solução: [7, 9, 0, 8, 1, 3, 8, 6, 1, 3] @ 4
('TwoPoints', 'UniformInt', 'Tournament')     |       4.74 s | Melhor solução: [6, 4, 2, 0, 3, 10, 8, 5, 9, 1] @ 0
('TwoPoints', 'UniformInt', 'Roulette')       |       8.44 s | Melhor solução: [7, 2, 

# CONCLUSÃO

Para cada um dos cenários solicitados (N = 10, 15 e 20), as configuração otimizadas são ligeiramente diferentes.  
Além disso, para cenários com menor complexidade e tamanho dos vetores de resposta, seria possível otimizar os parâmetros de tamanho de população e numero de gerações.  
A conclusão final é de que para cada cenário e aplicação, uma nova configuração é indicada para otimizar o algoritmo.  
Para cenários parecidos, os ganhos com diferentes configurações não ficam tão expressivos. 